# Szyfrowanie homomorficzne

### Zadanie 1
Dany jest szyfrogram otrzymany w wyniku działania algorytmu RSA (Textbook RSA encryption) na tekście jawnym o wartości 4. Na podstawie tego szyfroghramu wygeneruj fałszywy szyfrogram, który po odszyfrowaniu będzie równy 44. 

In [3]:
import random, sys, os

## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a   
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:   
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

def generateKey(keySize):
    print('1. Generujemy liczby p i q')
    p=49109 
    q=40639
    e = 65537 
    print('3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)')
    d=findModInverse(e, (p-1)*(q-1))
    n= p*q
    publicKey = (n, e)
    privateKey = (n, d)
    print('Klucz publiczny:', publicKey)
    print('Klucz prywatny:', privateKey)
    return (publicKey, privateKey)

public, private = generateKey(16)
  
print('Generujemy klucze publiczny i prywatny:', public, private)

def encrypt(data_number, modulus, exp):
    data_encrypted = pow(data_number, exp, modulus)
    return data_encrypted

def decrypt(data_number, modulus, exp):
    data_decrypted = pow(data_number, exp, modulus)
    return data_decrypted



1. Generujemy liczby p i q
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)
Klucz publiczny: (1995740651, 65537)
Klucz prywatny: (1995740651, 601158737)
Generujemy klucze publiczny i prywatny: (1995740651, 65537) (1995740651, 601158737)
350346157


4

In [7]:
# A^b

criptext1 = encrypt(4, public[0], public[1])
print("kryptogram:", criptext1)

criptext2 = encrypt(11, public[0], public[1])

criptext1 *= criptext2

decrypt1 = decrypt(criptext1, private[0], private[1])

print("odokodowna wiadomość:", decrypt1)

kryptogram: 350346157
odokodowna wiadomość: 44


## Kryptosystem Palliera



In [19]:
!pip install phe

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 KB 703.6 kB/s eta 0:00:001m945.7 kB/s eta 0:00:01


### Kryptosystem Pailliera

Q 1999 roku Pascal Paillier zaproponował kryptosystem zbliżony konstrukcją do RSA (bezpieczeństwo oparte o faktoryzację liczb całkowitych), które posiada własność szyfrowania homomorficznego dla operacji:
- dodawania
- mnożenia szyfrogramu przez jawny numer


#### Generowanie kluczy

1. Wybierz losowo dwie duże liczby pierwsze $p$ i $q$ tak aby $gcd(pq, (p-1)(q-1)) = 1$. 
2. Oblicz $n = pq$
3. Oblicz $\lambda = lcm(p-1,q-1)$ ($lcm$ --- Least Common Multiple, Najmniejsza Wspólna Wielokrotność)
4. Zdefiniuj funkcje $L(x) = \frac{x-1}{n}$
5. Wybierz losowo $g \in Z^*_{n^2}$ (liczba całkowita w zakresie 1 do $n^2$)
6. Oblicz odwrotność multiplikatywną $\mu = L(g^\lambda \bmod n^2))^{-1} bmod n$. Jeśli $\mu$ nie istnieje zacznij od nowa. 

Klucz publiczny ma postać: $pk = (n,g)$
Klucz prywatny ma postać: $sk = \lambda$ 

#### Szyfrowanie 

1. Tekstem jawnym jest liczba $m$ zakresu $0 \le m < n$.

2. Wybierz losową liczbę z zakresu $0 \le r < n$.

3. Oblicz szyfrogram $c= g^m \cdot r^n \bmod n^2$

#### Deszyfrowanie

1. Szyfrogram musi być liczbą z zakresu $0 < c < n^2$ 
2. Oblicz tekst jawny $m= L(c^\lambda \bmod n^2)\cdot \mu \bmod n$

### Homomorficzne własności schematu Pailliera

1. Dodawanie dwóch liczb:
$$D_{priv}(E_{pub}(m_1) \cdot E_{pub}(m_2) \bmod n^2)= m_1 + m_2 \bmod n$$

2. Mnożenie szyfrogramu przez liczbę:
$$D_{priv}(E_{pub}(m_1)^{m_2} \bmod n^2)= m_1 \cdot m_2 \bmod n$$

In [16]:
import phe  # https://pypi.org/project/phe/

from phe import paillier
import json
pub_key, priv_key = paillier.generate_paillier_keypair()
print(pub_key, priv_key)

enc1 = pub_key.encrypt(5)
enc2 = pub_key.encrypt(10)

print(enc1, enc2)

dec1 = priv_key.decrypt(enc1)
dec2 = priv_key.decrypt(enc2)

print(dec1, dec2)

enc3 = pub_key.encrypt(5)
dec3 = list()
dec3.append(priv_key.decrypt(enc3 + enc3))
dec3.append(priv_key.decrypt(enc3 * 5))
# dec3.append( priv_key.decrypt(enc3 * enc3))
# dec3.append(priv_key.decrypt(enc3 ** 5))
# dec3.append(priv_key.decrypt(enc3 ** enc3))
print(dec3)

<PaillierPublicKey 1056396212> <PaillierPrivateKey for <PaillierPublicKey 1056396212>>
<phe.paillier.EncryptedNumber object at 0x0000011255B99ED0> <phe.paillier.EncryptedNumber object at 0x0000011255CC7AD0>
5 10
[10, 25]


### Zadanie 2
Sprawdź empirycznie działanie homomorficznego szyfrowania Pailliera. Dlaczego to działa? Wykaż poprawność dodawania i mnożenia przez liczbę.   